In [ ]:
# Code to test the new smote_smfb function

In [ ]:
! pip install pyyaml

In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import gc
import os

## Modelling related packages
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.datasets import make_classification
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTEN
from sklearn.neighbors import KNeighborsClassifier


In [2]:
from generate_imbalanced_data import generate_imbalanced_data

In [4]:
n_rows = 500
imbalance_ratio = 0.05  # 10% minority class, 90% majority class
n_columns = 100
n_cols_imp_var = int(n_columns*0.4)  ## no. of top important variables to be chosen
no_of_nn = int ( ( n_rows - (2*imbalance_ratio * n_rows) ) / ( imbalance_ratio * n_rows) ) ## no. of nearest neighbours to get balanced data
flip_y = 0 ## fraction of samples class assigned randomly. Larger values introduce noise in the labels & make the classification task harder. 
class_sep = 0.2 ## Lower the number higher the overlap between boundaries of classes
informative_feature_perc = 0.2 ## Percentage of the informative features

In [5]:
full_data = generate_imbalanced_data(n_rows, imbalance_ratio, n_columns, flip_y, class_sep, informative_feature_perc)

/repos/smote_msfb/Tencent_experiments/generate_imbalanced_data.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minority_class_flipped['target'] = 0
/repos/smote_msfb/Tencent_experiments/generate_imbalanced_data.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  majority_class_flipped['target'] = 1


In [6]:
full_data.head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,target
0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0


In [7]:
import yaml

with open("/repos/smote_msfb/functions/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [8]:
if isinstance(config, dict):
    print("config is a dictionary ✅")
else:
    print("config is NOT a dictionary ❌")

config is a dictionary ✅


In [9]:
classification_model = LogisticRegression(penalty='l2', random_state=42)

In [10]:
sys.path.append('/repos/smote_msfb/functions')

from smote_msfb_cross_validation import smote_msfb_cross_validation
from smote_msfb import smote_msfb 
from get_hubness_score_minority_class import get_hubness_score_minority_class

In [11]:
full_data.iloc[:,:50].head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [12]:
full_data['target'].head(3)

0    0
1    0
2    0
Name: target, dtype: int64

In [13]:
y_train = full_data['target']
x_train = full_data.iloc[:,:10]

In [14]:
best_sampling_strategy = smote_msfb_cross_validation(x_train, y_train, config, classification_model)


Current minority/majority ratio ≈ 0.05
Testing sampling_strategy values: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

▶ Fold 1/4
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.1, ROC-AUC=0.2801
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.2, ROC-AUC=0.3711
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.3, ROC-AUC=0.3473
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.4, ROC-AUC=0.3725
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.5, ROC-AUC=0.3852
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.6, ROC-AUC=0.3922
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.7, ROC-AUC=0.3866
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.8, ROC-AUC=0.3922
Minority samples removed from anamoly detection : [323]
  - sampling_strategy=0.9, ROC-A

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ubun